<a href="https://colab.research.google.com/github/sumapnair/GenAI/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Based on this [medium article](https://medium.com/thedeephub/a-practical-guide-for-rag-and-few-shot-prompting-in-langchain-0b0e18dc9df5)**


In [ ]:
!pip install langchain-openai
!pip install langchain
!pip install langchain_community
!pip install langchain_core
!pip install gradio
!pip install unstructured
!pip install unstructured[pdf]
!pip install Chromadb
!pip install numpy==1.24.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 13.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 7.2 MB/s eta 0:00:00
  Created wheel for pypika: filename=PyPika-0.48.9-py2.py3-

In [ ]:
import os
from google.colab import userdata
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)
from operator import itemgetter

In [ ]:
# OpenAI API Key from collab secret
OPENAI_API_KEY = 'sk-proj-pCEo48ulCnXO2A5g3pWHT3BlbkFJuVX3y1AgN1GA6UqMypdm'

# initializing the LLM
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name="gpt-4",
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install poppler-utils
# this is to resolve the error 'PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH'

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
# To prevent TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.
! apt install tesseract-ocr
! apt install libtesseract-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libtesseract-dev is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
! pip install Pillow
! pip install pytesseract

In [ ]:
import pytesseract
from PIL import ImageEnhance, ImageFilter, Image

In [ ]:
print("Loading Data")
# This wil ltake awhile based on file size
# Follow these steps to import files in google drive to colab:
# https://saturncloud.io/blog/how-to-import-files-from-google-drive-to-colab/
loader = UnstructuredFileLoader('/content/drive/MyDrive/OracleUserGuide.pdf')
raw_doc = loader.load()


Loading Data


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# segmenting the document into segments

text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
texts = text_splitter.split_documents(raw_doc)

# Document Embedding with Chromadb

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
docsearch = Chroma.from_documents(texts, embeddings)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
# Connection to query with Chroma indexing using a retriever
retriever = docsearch.as_retriever(
    search_type="similarity",
    search_kwargs={'k':4}
)

# Function to add all docs returned by retriever

def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
docs = retriever.get_relevant_documents("tell me about oracle database")
for d in docs:
  print(d.page_content)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Oracle Applications provides you with easy–to–use search capabilities to access application information. You can quickly retrieve and review all available information in your database without having to remember the information displayed in windows, or print lengthy reports to see the data. Instead, you can simply run a search to obtain the information you want and review the data online in the same window used to enter the data.
Oracle provides powerful tools you can use to create, store, change, retrieve and maintain information in an Oracle database. But if you use Oracle tools like SQL*Plus to modify Oracle Applications data, you risk destroying the integrity of your data and you lose the ability to audit changes to your data.
Oracle develops and markets an integrated line of software products for database management, applications development, decision support, and office automation, as well as a complete family of financial, manufacturing, and human resource applications.
Oracle pr

In [ ]:
few_shot_examples = [
{"input":"""Could you please clarify the directions for Navigating Between Windows""",
"output":"""In a GUI environment, navigating between application windows can be inconvenient when dealing with many overlapping windows. To navigate between open windows, you can either click on the desired window or choose its name from the Window menu, which lists all currently open windows in Oracle Applications. This allows you to easily switch between windows without having to move or close others. By using the Window menu, you can quickly find and access the window you need."""},
{"input":"""What are these messages on screeen?""",
"output":"""When your application needs to provide a hint or basic error, it displays a short message. For more significant issues, an error dialog appears. A History button may also appear, allowing you to view details about the current error and recent errors. The application saves a history of your 5 most recent error messages, which are cleared when you change your responsibility. Error codes (e.g. FRM-, ORA-, APP-) help diagnose issues."""},
{"input":"""How do you translte records?""",
"output":"""To enter a translation for a record:

* Select the record and click the Translation icon in the Toolbar.
* In the Translations window, fields that can be translated are displayed in columns.
* Enter the translation for the appropriate fields.
* Click OK.

Note: The Translation icon is enabled in windows where translations are allowed."""},
{"input":"Can I use Shorthand Aliases in a Key Flexfield?",
"output":"Yes, you can, provided it is enabled. When using shorthand data entry for a key flexfield, a shorthand alias window appears where you can enter an alias, display a list of valid aliases, or cancel. If you enter a valid alias, the application closes the window and displays the equivalent concatenated segment values. If you enter a partial alias, the key flexfield Edit window appears with partial values inserted. If you enter an invalid alias, a list of valid aliases is displayed. You can also bypass shorthand entry by choosing OK or Cancel, which opens the key flexfield Edit window."}
]

few_shot_template = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}")
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=few_shot_template,
    examples=few_shot_examples,
)

print(few_shot_prompt.format())

Human: Could you please clarify the directions for Navigating Between Windows
AI: In a GUI environment, navigating between application windows can be inconvenient when dealing with many overlapping windows. To navigate between open windows, you can either click on the desired window or choose its name from the Window menu, which lists all currently open windows in Oracle Applications. This allows you to easily switch between windows without having to move or close others. By using the Window menu, you can quickly find and access the window you need.
Human: What are these messages on screeen?
AI: When your application needs to provide a hint or basic error, it displays a short message. For more significant issues, an error dialog appears. A History button may also appear, allowing you to view details about the current error and recent errors. The application saves a history of your 5 most recent error messages, which are cleared when you change your responsibility. Error codes (e.g. FRM

In [ ]:
# Creating prompt and LCEL
negotiate_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an Oracle expert. Do your best to advise users on how to use the Oracle E-Business Suite Application"),
    few_shot_prompt,
    ("user", "{question}"),
    ("user", "{context}")
])

# Langchain Expression Language to call our LLM using the prompt template above
# RAG chain
negotiate_chain = (
    {"context": itemgetter("question") | retriever | format_docs,
     "question": itemgetter("question")}
    | negotiate_prompt
    | llm
    | StrOutputParser()
    )

In [ ]:
negotiate_chain.invoke({"question":"How are concurrent requests handled in Oracle E-Biz suite?. Cite the sources"})


"manager starts running your concurrent request when:\n\n- It is the next highest priority request waiting to be run, and\n- A concurrent manager becomes available to run your request.\n\nOnce your request starts running, you are free to do any other work within Oracle Applications. You can even sign off, if you want to. When your request completes, the concurrent manager puts your request’s output in a location where you can view it, or sends your request’s output to a printer.\n\nThe number of requests that can run simultaneously depends on how your system administrator has set up concurrent processing. Your system administrator determines the number of concurrent processes that can run concurrently (hence the term concurrent processing), and the priority of all the requests you submit.\n\nSource: Oracle Applications User's Guide Release 12.2 (Document ID: E22953-08)"